In [1]:
import torch
# import torchvision
import torchtext
from torch import nn
from torch.utils.data import DataLoader
from perceiver_pytorch import Perceiver
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import itertools
from torch.utils.tensorboard import SummaryWriter
import os
from tqdm.notebook import tqdm
from datetime import datetime
from babi.babi_loader import BabiDataset, pad_collate

In [7]:
for task_id in range(1, 21):
    dset = BabiDataset(task_id, 
                       ds_path='/home/gabriel/Documents/datasets/bAbi/en/qa{}_*', 
                       vocab_path='/home/gabriel/Documents/datasets/bAbi/en/babi{}_vocab.pkl')
    vocab_size = len(dset.QA.VOCAB)
    dset.set_mode('train')
    train_loader = DataLoader(
        dset, batch_size=100, shuffle=True, collate_fn=pad_collate
    )
    for x in train_loader:
        print(task_id, x[0].shape, x[1].shape, x[2].shape)
        print(x[0])
        print(x[1])
        print(x[2])
        break

1 torch.Size([100, 10, 8]) torch.Size([100, 4]) torch.Size([100])
tensor([[[ 2., 19.,  4.,  ...,  7.,  1.,  0.],
         [ 2.,  9., 14.,  ..., 21.,  7.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[15.,  3.,  4.,  ...,  7.,  1.,  0.],
         [13., 18.,  4.,  ...,  7.,  1.,  0.],
         [13., 19.,  4.,  ...,  7.,  1.,  0.],
         ...,
         [ 2.,  9., 14.,  ..., 20.,  7.,  1.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[13., 19.,  4.,  ...,  7.,  1.,  0.],
         [15.,  9.,  4.,  ...,  7.,  1.,  0.],
         [15.,  9., 14.,  ...,  6.,  7.,  1.],
         ...,
         [15.,  3.,  4.,  ...,  7.,  1.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[13.,  9., 14.,  

3 torch.Size([100, 70, 8]) torch.Size([100, 8]) torch.Size([100])
tensor([[[14., 36.,  5.,  ...,  7.,  1.,  0.],
         [ 2., 27.,  4.,  ...,  7.,  1.,  0.],
         [ 8.,  3.,  4.,  ...,  7.,  1.,  0.],
         ...,
         [ 2., 27.,  4.,  ...,  7.,  1.,  0.],
         [ 8., 33.,  5.,  ...,  1.,  0.,  0.],
         [29., 15., 17.,  ..., 18.,  7.,  1.]],

        [[29., 27.,  4.,  ...,  7.,  1.,  0.],
         [ 2.,  3.,  4.,  ...,  7.,  1.,  0.],
         [ 2.,  3.,  4.,  ...,  7.,  1.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 8., 27.,  4.,  ...,  7.,  1.,  0.],
         [ 8., 15.,  4.,  ...,  7.,  1.,  0.],
         [ 2., 27.,  4.,  ...,  7.,  1.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[14., 27.,  4.,  

5 torch.Size([100, 70, 8]) torch.Size([100, 8]) torch.Size([100])
tensor([[[15., 29.,  4.,  ...,  7.,  1.,  0.],
         [31.,  3.,  4.,  ...,  7.,  1.,  0.],
         [31., 29.,  4.,  ...,  7.,  1.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 2.,  3.,  4.,  ...,  7.,  1.,  0.],
         [15., 28.,  4.,  ...,  7.,  1.,  0.],
         [31., 12., 21.,  ..., 27.,  7.,  1.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 2., 28.,  4.,  ...,  7.,  1.,  0.],
         [31.,  3.,  4.,  ...,  7.,  1.,  0.],
         [15.,  3.,  4.,  ...,  7.,  1.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[15., 39.,  5.,  

7 torch.Size([100, 22, 8]) torch.Size([100, 7]) torch.Size([100])
tensor([[[25., 32., 33.,  ..., 16.,  7.,  1.],
         [ 8., 27.,  4.,  ...,  7.,  1.,  0.],
         [25., 42.,  5.,  ...,  8.,  7.,  1.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 2., 14.,  5.,  ...,  7.,  1.,  0.],
         [ 2., 29.,  5.,  ...,  7.,  1.,  0.],
         [25.,  3.,  4.,  ...,  7.,  1.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 2., 12.,  4.,  ...,  7.,  1.,  0.],
         [ 2.,  9.,  4.,  ...,  7.,  1.,  0.],
         [ 8., 12.,  4.,  ...,  7.,  1.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[25., 12.,  4.,  

9 torch.Size([100, 10, 9]) torch.Size([100, 6]) torch.Size([100])
tensor([[[ 2., 25., 12.,  ...,  1.,  0.,  0.],
         [10., 11., 12.,  ...,  1.,  0.,  0.],
         [ 2., 11., 12.,  ...,  1.,  0.,  0.],
         ...,
         [19.,  3., 16.,  ...,  9.,  1.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 2.,  3.,  6.,  ...,  1.,  0.,  0.],
         [ 2.,  3.,  4.,  ...,  8.,  9.,  1.],
         [19.,  3., 16.,  ...,  9.,  1.,  0.],
         ...,
         [19.,  3.,  6.,  ...,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 2.,  3.,  6.,  ...,  1.,  0.,  0.],
         [14.,  3.,  6.,  ...,  1.,  0.,  0.],
         [10.,  3., 16.,  ...,  9.,  1.,  0.],
         ...,
         [10., 11., 12.,  ...,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[10.,  3.,  6.,  

11 torch.Size([100, 10, 10]) torch.Size([100, 4]) torch.Size([100])
tensor([[[26., 22.,  5.,  ...,  0.,  0.,  0.],
         [24., 10., 11.,  ...,  8.,  1.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[26., 22.,  5.,  ...,  0.,  0.,  0.],
         [ 9., 10., 11.,  ...,  8.,  1.,  0.],
         [15., 16.,  5.,  ...,  0.,  0.,  0.],
         ...,
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[15., 16.,  5.,  ...,  0.,  0.,  0.],
         [27., 19.,  3.,  ...,  1.,  0.,  0.],
         [26.,  3.,  5.,  ...,  0.,  0.,  0.],
         ...,
         [18., 11., 23.,  ...,  1.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        ...,

        [[ 2., 16.,  5.,

13 torch.Size([100, 10, 10]) torch.Size([100, 4]) torch.Size([100])
tensor([[[ 4.,  3.,  2.,  ...,  9.,  1.,  0.],
         [23., 24., 11.,  ...,  9.,  1.,  0.],
         [ 2.,  3., 17.,  ...,  9.,  1.,  0.],
         ...,
         [26., 24., 11.,  ...,  9.,  1.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]],

        [[ 4.,  3.,  2.,  ...,  9.,  1.,  0.],
         [26., 24., 11.,  ...,  9.,  1.,  0.],
         [ 2.,  3.,  4.,  ...,  9.,  1.,  0.],
         ...,
         [23., 24., 11.,  ...,  9.,  1.,  0.],
         [ 4.,  3., 17.,  ...,  9.,  1.,  0.],
         [27., 11.,  5.,  ...,  1.,  0.,  0.]],

        [[ 4.,  3.,  2.,  ...,  9.,  1.,  0.],
         [10., 11.,  5.,  ...,  9.,  1.,  0.],
         [ 2.,  3.,  4.,  ..., 19.,  9.,  1.],
         ...,
         [27., 11.,  5.,  ...,  1.,  0.,  0.],
         [16.,  3.,  2.,  ...,  8.,  9.,  1.],
         [10., 11.,  5.,  ...,  9.,  1.,  0.]],

        ...,

        [[17.,  3.,  2.,

15 torch.Size([100, 8, 7]) torch.Size([100, 6]) torch.Size([100])
tensor([[[ 2.,  3.,  4.,  ...,  6.,  7.,  1.],
         [16.,  9., 10.,  ...,  7.,  1.,  0.],
         [ 6.,  3.,  4.,  ...,  2.,  7.,  1.],
         ...,
         [15.,  9., 10.,  ...,  7.,  1.,  0.],
         [ 8.,  9., 10.,  ...,  7.,  1.,  0.],
         [14.,  9., 10.,  ...,  7.,  1.,  0.]],

        [[13.,  3.,  4.,  ...,  2.,  7.,  1.],
         [12.,  3.,  4.,  ...,  2.,  7.,  1.],
         [ 6.,  3.,  4.,  ..., 13.,  7.,  1.],
         ...,
         [ 8.,  9., 10.,  ...,  7.,  1.,  0.],
         [14.,  9., 10.,  ...,  7.,  1.,  0.],
         [15.,  9., 10.,  ...,  7.,  1.,  0.]],

        [[ 2.,  3.,  4.,  ...,  6.,  7.,  1.],
         [12.,  3.,  4.,  ...,  6.,  7.,  1.],
         [16.,  9., 10.,  ...,  7.,  1.,  0.],
         ...,
         [13.,  3.,  4.,  ...,  2.,  7.,  1.],
         [14.,  9., 10.,  ...,  7.,  1.,  0.],
         [15.,  9., 10.,  ...,  7.,  1.,  0.]],

        ...,

        [[ 6.,  3.,  4.,  

17 torch.Size([100, 2, 13]) torch.Size([100, 12]) torch.Size([100])
tensor([[[ 2., 20., 10.,  ..., 10.,  8.,  1.],
         [ 2.,  9., 10.,  ...,  0.,  0.,  0.]],

        [[ 2.,  6.,  7.,  ...,  8.,  1.,  0.],
         [ 2.,  6.,  7.,  ...,  0.,  0.,  0.]],

        [[ 2.,  6.,  7.,  ..., 10.,  8.,  1.],
         [ 2., 18., 10.,  ...,  0.,  0.,  0.]],

        ...,

        [[ 2., 18., 10.,  ..., 19.,  8.,  1.],
         [ 2., 18., 10.,  ..., 10.,  8.,  1.]],

        [[ 2.,  3.,  4.,  ...,  8.,  1.,  0.],
         [ 2.,  6.,  7.,  ..., 10.,  8.,  1.]],

        [[ 2.,  9., 10.,  ...,  7.,  8.,  1.],
         [ 2.,  9., 10.,  ..., 10.,  8.,  1.]]], dtype=torch.float64)
tensor([[ 4,  2,  6,  ..., 20, 10,  1],
        [ 4,  2,  9,  ...,  0,  0,  0],
        [ 4,  2, 18,  ...,  6,  7,  1],
        ...,
        [ 4,  2, 18,  ...,  0,  0,  0],
        [ 4,  2,  9,  ...,  0,  0,  0],
        [ 4,  2,  6,  ..., 18, 10,  1]])
tensor([17, 17, 15, 15, 17, 17, 17, 17, 17, 15, 17, 15, 15, 15, 15,

19 torch.Size([100, 5, 9]) torch.Size([100, 11]) torch.Size([100])
tensor([[[ 2.,  7.,  4.,  ..., 13.,  8.,  1.],
         [ 2., 13.,  4.,  ..., 14.,  8.,  1.],
         [ 2.,  3.,  4.,  ..., 13.,  8.,  1.],
         [ 2., 11.,  4.,  ...,  3.,  8.,  1.],
         [ 2., 14.,  4.,  ...,  9.,  8.,  1.]],

        [[ 2., 13.,  4.,  ..., 11.,  8.,  1.],
         [ 2.,  3.,  4.,  ..., 11.,  8.,  1.],
         [ 2.,  7.,  4.,  ...,  9.,  8.,  1.],
         [ 2.,  3.,  4.,  ...,  9.,  8.,  1.],
         [ 2.,  9.,  4.,  ..., 14.,  8.,  1.]],

        [[ 2.,  9.,  4.,  ..., 11.,  8.,  1.],
         [ 2., 14.,  4.,  ..., 11.,  8.,  1.],
         [ 2.,  3.,  4.,  ..., 11.,  8.,  1.],
         [ 2., 13.,  4.,  ...,  9.,  8.,  1.],
         [ 2., 13.,  4.,  ...,  7.,  8.,  1.]],

        ...,

        [[ 2.,  7.,  4.,  ..., 14.,  8.,  1.],
         [ 2.,  9.,  4.,  ..., 11.,  8.,  1.],
         [ 2.,  9.,  4.,  ..., 13.,  8.,  1.],
         [ 2., 11.,  4.,  ...,  7.,  8.,  1.],
         [ 2.,  3., 

In [4]:
# bAbI = 